In [1]:
import pandas as pd

In [2]:
# 1、获取数据
titanic = pd.read_csv("titanic.csv")

In [3]:
titanic.head()

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


In [4]:
# 筛序特征值和目标值
x = titanic[["pclass", "age", "sex"]]
y = titanic["survived"]

In [5]:
x.head()

,pclass,age,sex
0,1st,29.0000,female
1,1st,2.0000,female
2,1st,30.0000,male
3,1st,25.0000,female
4,1st,0.9167,male


In [6]:
y.head()

0    1
1    0
2    0
3    0
4    1
Name: survived, dtype: int64

In [7]:
# 2、数据处理
# 1）缺失值处理
x["age"].fillna(x["age"].mean(), inplace=True)

/var/folders/3t/l_d637_s7w9g7p_8jx595_l00000gn/T/ipykernel_6857/2701345224.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["age"].fillna(x["age"].mean(), inplace=True)


In [8]:
x

,pclass,age,sex
0,1st,29.000000,female
1,1st,2.000000,female
2,1st,30.000000,male
3,1st,25.000000,female
4,1st,0.916700,male
...,...,...,...
1308,3rd,31.194181,male
1309,3rd,31.194181,male
1310,3rd,31.194181,male
1311,3rd,31.194181,female


In [9]:
# 2）转化成字典
x = x.to_dict(orient="records")

In [10]:
from sklearn.model_selection import train_test_split

# 3、数据集划分
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)

In [11]:
# 4、字典特征抽取
from sklearn.feature_extraction import DictVectorizer

In [12]:
transfer = DictVectorizer()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [13]:
from sklearn.tree import DecisionTreeClassifier

# 3. 决策树预估器
estimator = DecisionTreeClassifier(criterion="entropy", max_depth=5)
estimator.fit(x_train, y_train)

# 4. 模型评估
# 方法1：直接比对真实值和预测值
y_predict = estimator.predict(x_test)
print("y_predict:\n", y_predict)
# print("直接比对真实值和预测值：\n", y_test == y_predict)

# 方法2：计算准确率
# 先用训练集造出模型再用测试集来预测并得到准确率
score = estimator.score(x_test, y_test)
print("决策树准确率为：\n", score)

y_predict:
 [0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0
 0 0 0 1 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0
 0 1 1 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1]
决策树准确率为：
 0.790273556231003


### 随机森林对泰坦尼克号乘客的生存进行预测

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
from sklearn.model_selection import GridSearchCV

estimator = RandomForestClassifier()

param_dict = {"n_estimators": [120, 200, 300, 500, 800, 1200],
              "max_depth": [5, 8, 15, 25, 30]}
estimator = GridSearchCV(estimator, param_grid=param_dict, cv=3)
estimator.fit(x_train, y_train)

# 先用训练集造出模型再用测试集来预测并得到准确率
score = estimator.score(x_test, y_test)
print("随机森林准确率为：\n", score)

随机森林准确率为：
 0.7872340425531915
